In [44]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [45]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [46]:
# specify the url strings for the company's job posting website
url = 'https://nvidia.wd5.myworkdayjobs.com/NVIDIAExternalCareerSite'
driver = webdriver.Chrome()
driver.implicitly_wait(2)
driver.get(url)

In [47]:
# get job titles and links for each page and click the next button to go to the next page until no more
job_title=[]
job_link=[]

next_button = driver.find_element('xpath', '//*[@aria-label="next"]')  
while next_button:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    job_title.extend([td.text for td in soup.findAll("a", {"data-automation-id": "jobTitle"})])
    job_link.extend(['https://nvidia.wd5.myworkdayjobs.com' + td['href'] for td in soup.findAll("a", {"data-automation-id": "jobTitle"})])
    try:
        next_button.click()
        time.sleep(1)
    except: break

In [48]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

## Extract job descriptions and qualifications for each job

In [49]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''

        text_pattern_descrp = re.compile("ll be doing", re.IGNORECASE)
        text_pattern_qual_1 = re.compile("what we need to see", re.IGNORECASE)        
        text_pattern_qual_2 = re.compile("ways to stand out from the crowd", re.IGNORECASE)  
        
        try:
          tag = soup.find("b", text=text_pattern_qual_1).findNext("ul")
          s = s + tag.text
        except: pass
        
        try:
          tag = soup.find('b', text=text_pattern_qual_2).findNext("ul")
          s = s + tag.text
        except: pass

        try:
          tag = soup.find("b", text=text_pattern_descrp).findNext("ul")
          d = tag.text
        except: pass
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [50]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)
print(len(qual), len(descrp))

660 660


In [51]:
def remove_space(s):
    s_new = s.replace('\n', '')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new


# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

In [52]:
# do some cleaning
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

#create a dataframe from qualifications and descriptions
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Nvidia'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [53]:
# drop the empty Qualification entries
empty_mini_q = get_empty(df, 'QUALIFICATIONS')
df.drop(empty_mini_q, inplace=True)

#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from Nvidia.".format(df_nodup.shape[0]))

# save the output file
df_nodup.to_csv('nvidia_all_jobs_cleaned.csv')

There are 627 jobs from Nvidia.
